In [35]:
from io import BytesIO
import pycurl,timeit
from multiprocessing import Process
from multiprocessing import Manager

# Performance with proxy

In [36]:
def proxy_latency(prox_type,host,port,target,return_list):
    buffer = BytesIO()
    c = pycurl.Curl()
    c.setopt(pycurl.PROXY, f"{host}:{port}")
    c.setopt(pycurl.URL, target)
    c.setopt(c.WRITEDATA, buffer)
    ts = timeit.default_timer()
    c.perform()
    
    m = {}
    m['prox_type'] = prox_type
    m['target'] = target
    m['total-time'] = c.getinfo(pycurl.TOTAL_TIME)
    m['namelookup-time'] = c.getinfo(pycurl.NAMELOOKUP_TIME)
    m['connect-time'] = c.getinfo(pycurl.CONNECT_TIME)
    m['pretransfer-time'] = c.getinfo(pycurl.PRETRANSFER_TIME)
    m['starttransfer-time'] = c.getinfo(pycurl.STARTTRANSFER_TIME)
    
    return_list.append(m)
    
    c.close()
    
    return m


# Performance without proxy

In [37]:
def normal_latency(port,target,return_list):
    buffer = BytesIO()
    c = pycurl.Curl()
    c.setopt(pycurl.URL, target)
    c.setopt(c.WRITEDATA, buffer)
    ts = timeit.default_timer()
    c.perform()
    
    m = {}
    m['prox_type'] = "N/A"
    m['target'] = target
    m['total-time'] = c.getinfo(pycurl.TOTAL_TIME)
    m['namelookup-time'] = c.getinfo(pycurl.NAMELOOKUP_TIME)
    m['connect-time'] = c.getinfo(pycurl.CONNECT_TIME)
    m['pretransfer-time'] = c.getinfo(pycurl.PRETRANSFER_TIME)
    m['starttransfer-time'] = c.getinfo(pycurl.STARTTRANSFER_TIME)
    
    return_list.append(m)
    
    c.close()
    

# Setup

In [38]:
host = "130.64.148.78"
port = "8080"
range_k = 100



# target = "https://www.reddit.com/"

target = "http://www.cs.cmu.edu/~prs/bio.html"


# target = "http://www.cs.tufts.edu/comp/112/"

# Without proxy

## 1. One time

In [39]:
return_list = []
normal_latency(port,target,return_list)
print(return_list[0])

{'prox_type': 'N/A', 'target': 'http://www.cs.cmu.edu/~prs/bio.html', 'total-time': 0.059507, 'namelookup-time': 0.00348, 'connect-time': 0.020911, 'pretransfer-time': 0.020954, 'starttransfer-time': 0.040556}


## 2. Average time (100 request)

In [40]:
total = 0
return_list = []
for n in range(range_k):
    result = normal_latency(port,target,return_list)

for result in return_list: 
    total+= result["total-time"]
    
total/range_k

0.09092622999999994

# Our project - select + pthread

In [41]:
prox_type = "select"

## 1. One time without cache

In [42]:
return_list = []
proxy_latency(prox_type,host,port,target,return_list)
print(return_list[0])

{'prox_type': 'select', 'target': 'http://www.cs.cmu.edu/~prs/bio.html', 'total-time': 0.058302, 'namelookup-time': 5e-05, 'connect-time': 0.000308, 'pretransfer-time': 0.000374, 'starttransfer-time': 0.05823}


## 2. Average time with cache (100 reqests)

In [44]:
total = 0
return_list = []
for n in range(range_k):
    proxy_latency(prox_type,host,port,target,return_list)

    
for result in return_list: 
    total+= result["total-time"]
    
total/range_k

0.00045407000000000005

# Fully fork version - without cache

In [132]:
prox_type = "fork"


In [133]:
total = 0

for n in range(range_k):
    result = proxy_latency(prox_type,host,port,target)
    total+= result["total-time"]
    
total/range_k

0.07034185000000004

# Fully pthread version - without cache

In [134]:
prox_type = "pthread"

In [135]:
total = 0

for n in range(range_k):
    result = proxy_latency(prox_type,host,port,target)
    total+= result["total-time"]
    
total/range_k

0.11215383000000007

# Simulate multi users

In [56]:
manager = Manager()
return_list = manager.list()
jobs = []
total = 0

for i in range(range_k):
    p = Process(target=normal_latency,args=(port,target,return_list))
    jobs.append(p)
    p.start()

for proc in jobs:
    proc.join()

for item in return_list:
    for element in item:
        print(element["total-time"])

total/range_k    

TypeError: string indices must be integers

In [54]:
manager2 = Manager()
return_list2 = manager.list()
jobs2 = []
total2 = 0

for i in range(range_k):
    p2 = Process(target=proxy_latency,args=(prox_type,host,port,target,return_list2))
    jobs2.append(p)
    p2.start()

for proc in jobs2:
    proc.join()

for item in return_list:
    for element in item:
        total2+= element["total-time"]

total/range_k  

0.0013720000000000108